In [ ]:
!pip install langchain
!pip install torch
!pip install accelerate
!pip install sentence-transformers
!pip install streamlit
!pip install streamlit-chat
!pip install faiss-cpu
!pip install tiktoken
!pip install huggingface-hub
!pip install pypdf
!pip install llama-cpp-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 15.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinu

In [ ]:
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.9 MB/s eta 0:00:00


In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import PyPDFLoader
import os
import tempfile

In [ ]:
def initialize_session_state():
    session_state = {
        "history": [],
        "generated": ["Hello! Ask me anything about 🤖"],
        "past": ["Hey! 👋"]
    }
    return session_state


In [ ]:
def conversation_chat(query, chain, session_state):
    result = chain({"question": query, "chat_history": session_state["history"]})
    session_state["history"].append((query, result["answer"]))
    return result["answer"]


In [ ]:
def display_chat_history(chain, session_state):
    user_input = input("Enter your question: ")
    if not user_input:
        return

    print(f"Repeating question '{user_input}' 10 times:")

    for _ in range(10):
        output = conversation_chat(user_input, chain, session_state)
        print("Bot:", output)



In [ ]:
from langchain.llms import DeepInfra


In [ ]:
os.environ["DEEPINFRA_API_TOKEN"] = "ohnGw99RAl4ZyJ6zI8gb7RpMGEZ30yHt"

In [ ]:
from langchain.llms import DeepInfra
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

def create_conversational_chain(vector_store):
    local_llm = DeepInfra(model_id="mistralai/Mistral-7B-Instruct-v0.1",
        #streaming=True,
        #temperature=0.75,
        #top_p=1,
        #verbose=True,
        #n_ctx=4096,
    )

    local_llm.model_kwargs = {
        "streaming":True,
        "temperature": 0.75,
        "top_p":1,
        "verbose":True,
        "n_ctx" : 4096,
    }

    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    chain = ConversationalRetrievalChain.from_llm(
        llm=local_llm,
        chain_type="stuff",
        retriever=vector_store.as_retriever(search_kwargs={"k": 2}),
        memory=memory
    )
    return chain

In [ ]:
import re
def clean_text(text):
    cleaned_text = text  # Initial cleaning steps if needed
    cleaned_text = cleaned_text.replace('\n', ' ')  # Replace newline characters with spaces
    cleaned_text = re.sub(r'[^\w\s]', '', cleaned_text)  # Remove symbols if necessary
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()  # Remove extra spaces
    cleaned_text = cleaned_text.replace('•', '')  # Remove bullet points
    cleaned_text = cleaned_text.replace('–', '-')  # Replace en-dashes with hyphens
    return cleaned_text


In [ ]:
from collections import namedtuple
Document = namedtuple('Document', ['page_content', 'metadata'])

In [ ]:

#clean pdf
from collections import namedtuple

# Define a named tuple for document representation
Document = namedtuple('Document', ['page_content', 'metadata'])

def main():
    session_state = initialize_session_state()
    print("ChatBot using Mistral-7B-Instruct LLM :books:")

    uploaded_files = ["/content/2023.pdf"]

    if uploaded_files:
        text = []
        for file_path in uploaded_files:
            with open(file_path, "rb") as f:
                file_contents = f.read()
            file_extension = os.path.splitext(file_path)[1]
            with tempfile.NamedTemporaryFile(delete=False) as temp_file:
                temp_file.write(file_contents)
                temp_file_path = temp_file.name
                print("loading: ", file_path)
            loader = None
            if file_extension == ".pdf":
                loader = PyPDFLoader(temp_file_path)

            if loader:
                text.extend(loader.load())
                os.remove(temp_file_path)

        # Clean up the text and remove newlines
        cleaned_text = [clean_text(page.page_content.replace('\n', ' ')) for page in text]

        # Create documents with metadata for splitting
        documents = [Document(page_content=cleaned_page, metadata={'source': '/tmp/tempfile'}) for cleaned_page in cleaned_text]

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=10000, chunk_overlap=20
        )
        text_chunks = text_splitter.split_documents(documents)

        print("Cleaned and split chunks:\n", text_chunks)

        embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2",
            model_kwargs={"device": "cpu"},
        )

        vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)

        chain = create_conversational_chain(vector_store)

        display_chat_history(chain, session_state)

if __name__ == "__main__":
    main()


ChatBot using Mistral-7B-Instruct LLM :books:
loading:  /content/2023.pdf
Cleaned and split chunks:
 [Document(page_content='Faculty of Information Technology University of Moratuwa Student Handbook Batch 22', metadata={'source': '/tmp/tempfile'}), Document(page_content='Faculty of Information Technology University of Moratuwa Student Handbook Batch 22 Bachelor of Science Honours in Information Technology Bachelor of Science Honours in Information Technology Management Bachelor of Science Honours in Artificial Intelligence', metadata={'source': '/tmp/tempfile'}), Document(page_content='CONTENT MESSAGE FROM THE DEAN 1 MESSAGE FROM THE AC TING REGISTRAR 2 MESSAGE FROM THE LIBRARIAN 3 MESSAGE FROM THE HEAD DEPARTMENT OF INFORMATION TECHNOLOGY 4 MESSAGE FROM THE HEAD DEPARTMENT OF INTERDISCIPLINARY STUDIES 5 MESSAGE FROM THE HEAD DEPARTMENT OF COMPUTATIONAL MATHEMATICS 6 MESSAGE FROM THE CHIEF STUDENT COUNSELLOR 7 MESSAGE FROM THE DIRECTOR CAREER GUIDANCE UNIT 8 THE UNIVERSITY OF MORATUWA 

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Enter your question: If a student miss an examination whom should he/she inform?
Repeating question 'If a student miss an examination whom should he/she inform?' 10 times:
Bot:  A student should inform the Senior Assistant Registrar Examinations within 7 days if he/she misses an examination.
Bot:   A student should inform the RegistrarSAR Examinations within 7 days if he/she misses an examination.
Bot:   If a student misses an examination, he/she should inform the RegistrarSAR Examinations within 7 days.
Bot:  A student should inform the RegistrarSAR Examination within 7 days and submit a medical certificate issued by a Government Hospital supported by the diagnosis card receipt of payment for the medical certificate results of investigations carried out such as blood test etc if any and prescription forms for the drug treatment given. If no diagnosis card is available, other documents mentioned above are essential. If a student misses only one paper, he/she may submit a Medical Certif

In [ ]:
#what i had first created
'''def main():
    session_state=initialize_session_state()
    print("ChatBot using Mistral-7B-Instruct LLM :books:")

    uploaded_files = ["/content/2023.pdf"]

    if uploaded_files:
        text = []
        for file_path in uploaded_files:
            with open(file_path, "rb") as f:
                file_contents = f.read()
            file_extension = os.path.splitext(file_path)[1]
            with tempfile.NamedTemporaryFile(delete=False) as temp_file:
                temp_file.write(file_contents)
                temp_file_path = temp_file.name
                print("loading: ", file_path)
            loader = None
            if file_extension == ".pdf":
                loader = PyPDFLoader(temp_file_path)

            if loader:
                text.extend(loader.load())
                os.remove(temp_file_path)

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=10000, chunk_overlap=20
        )
        text_chunks = text_splitter.split_documents(text)

        print("chunks:\n", text_chunks)

        embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2",
            model_kwargs={"device": "cpu"},
        )

        print("embeddings:\n", embeddings)

        vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)

        chain = create_conversational_chain(vector_store)

        display_chat_history(chain,session_state)


if __name__ == "__main__":
    main()
    '''